# Use-case : Drugs having a putative effect on a given metabolism

## Goal

Given a metabolic network that could be loaded as a cobra model (eq. in sbml format for instance), derive a list of genes that regulates directly the network's metabolic reactions (later, this list is refered as Targets).
Then, this list is used to find the Drugs having a direct impact on the metabolism (from the KeggDrugs repository).
pyBravo is then used to reconstruct the upstream gene regulatory network of the Targets genes. This provides a bigger list of direct and undirect regulators which allows to find the Drugs having a indirect impact on the metabolism.

Notice that when the metabolic network is provided with an objective function (such as the biomass production), a single reaction deletion analysis is first performed to reduce the size of the Targets gene list and to focus on the genes having an effect on the objective value.

## Step 1 : Extracting a target gene list from iLiverCancer Metabolic network


### 0 - [Facultative step] installing the required libraries (not required if the environment is prepared in advance)

In [1]:
!pip3 install cobra
!pip3 install python-libsbml

     |████████████████████████████████| 2.4MB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 133kB 11.4MB/s eta 0:00:01
     |████████████████████████████████| 829kB 11.2MB/s eta 0:00:01
     |████████████████████████████████| 1.6MB 11.5MB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 11.7MB/s eta 0:00:01
     |████████████████████████████████| 112kB 12.0MB/s eta 0:00:01
     |████████████████████████████████| 10.4MB 9.9MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/b2/34/bfcb43cc0ba81f527bc4f40ef41ba2ff4080e047acb0586b56b3d017ace4/zipp-3.1.0-py3-none-any.whl
     |████████████████████████████████| 5.8MB 10.6MB/s eta 0:00:01
     |████████████████████████████████| 153kB 11.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
     |████████████████████████████████| 522kB 11.3MB/s eta 0:00:01
  Created wheel 

### 1 - importing the required python libraries

In [2]:
import cobra
import json, io

### 2 - Setting parameters and name/id mappings

In [4]:
Param_SBMLfile = "../models/iLiverCancer1715.xml"
#Param_Percentage = 1.1 # Complete Target list (does not require any objective function)
Param_Percentage = 0.95 # Restriction of the gene such that, once deleted, reduce the objective value for more that 5%

filenameCorrespondanceEnsemblGenenames = '../tables/Correspondance_Ensembl_GeneNames_by_SPARQL.json'
filenameCorrespondanceHMREnsembl = "../tables/Correspondance_HMR_Ensembl.csv"


# Table HMR (Human Model Reaction) to Ensembl correspondance
fd = open(filenameCorrespondanceHMREnsembl,"r")

lines=fd.readlines()

HMR_to_Ensembl = dict()
for g in lines:
    t=g.split(";")
    t[1]=(t[1].replace("\n",""))
    HMR_to_Ensembl[t[0]]=t[1].split("|")
    
fd.close()


# Table Ensembl to gene names (HGNC symbols)
with open(filenameCorrespondanceEnsemblGenenames) as data_file:
    genelist = json.load(data_file)


GenenameToEnsembl=dict()
for gid in genelist:
    genename=genelist[gid][0]
    if not (GenenameToEnsembl.get(genename)):
        GenenameToEnsembl[genename]=list()
        GenenameToEnsembl[genename].append(gid)

GeneAltNameToName=dict()
for gid in genelist:
    genename=genelist[gid][0]
    GeneAltNameToName[genename]=genename
    for i in range(1,len(genelist[gid])):
        GeneAltNameToName[genelist[gid][i]]=genename


### 4 - reading the cobra model

In [7]:
# Note that the syntax of iLiverCancer1715.xml has been slightly modified to 
# conform to recent SBML parsers

model = cobra.io.read_sbml_model(Param_SBMLfile)

# Required for MetabolicAtlas models : set the objective function
model.reactions.CancerBiomass_OF.objective_coefficient=1

Model does not contain SBML fbc package information.
SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
urn:miriam:lipidmaps:LMFA01030283 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00003c "(10Z)-heptadecenoic acid">
urn:miriam:lipidmaps:LMFA01030283 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00003r "(10Z)-heptadecenoic acid">
urn:miriam:lipidmaps:LMFA01030283 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00003s "(10Z)-heptade

urn:miriam:lipidmaps:LMFA01030367 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00017s "(13Z)-eicosenoic acid">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00018c "(13Z)-eicosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00018m "(13Z)-eicosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00018p "(13Z)-eicosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00018r "(13Z)-eicosenoyl-CoA">
urn:miriam:lipidmaps:LMFA01030290 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula inste

Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00038c "(2E)-decenoyl-[ACP]">
urn:miriam:obo.chebi:CHEBI:10723 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00039m "(2E)-decenoyl-CoA">
urn:miriam:obo.chebi:CHEBI:10723 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00039p "(2E)-decenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00040c "(2E)-docosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00040m "(2E)-docosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species

urn:miriam:obo.chebi:CHEBI:27721 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00066p "(2E)-tetradecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00067c "(2E)-tricosenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00068c "(2E)-tridecenoyl-[ACP]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00069m "(2E)-tridecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00070c "(2E)-undecenoyl-[ACP]">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00071m "(2E)-undecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Spec

urn:miriam:obo.chebi:CHEBI:65132 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00095r "(4Z,7Z,10Z,13Z,16Z,19Z)-docosahexaenoyl-CoA">
urn:miriam:kegg.compound:C03740 does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00097s "(5-L-glutamyl)-L-amino acid">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00099m "(5Z)-dodecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00099p "(5Z)-dodecenoyl-CoA">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_m00100c "(5Z,8Z,11Z)-eicosatrienoylcarnitine">
Use of FORMULA in the notes element is dis

...

Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9156 "dITP">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9159 "thiamin[x]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9159 "thiamin[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9160 "hydrogen-cyanide">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9160 "hydrogen-cyanide">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9161 "homoserine[x]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9161 "homoserine[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9165 "oxalate[x]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9165 "oxalate[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9166 "chitin-component">'
Missing upper flux bound set to '1000.0' for  r

Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9279 "cys-gly">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9280 "cholate">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9280 "cholate">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9281 "glycocholate[x]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9281 "glycocholate[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9282 "taurocholate[x]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9282 "taurocholate[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9283 "glycochenodeoxycholate[x]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9283 "glycochenodeoxycholate[x]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9284 "taurochenodeoxycholate">'
Missing uppe

Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9379 "nifedipine">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9383 "O2-">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9383 "O2-">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9386 "hydroxide">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9386 "hydroxide">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9393 "progesterone">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9393 "progesterone">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9394 "D-proline">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_9394 "D-proline">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_9396 "prostaglandin E1">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_

Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_2 "4-hydroxyphenylpyruvate[c]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_2 "4-hydroxyphenylpyruvate[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_3 "3-hydroxyanthranilate[c]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_3 "3-hydroxyanthranilate[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_4 "3-phospho-D-glycerate[c]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_4 "3-phospho-D-glycerate[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_5 "4-hydroxybenzoate[c]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_5 "4-hydroxybenzoate[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_6 "trans-4-hydroxy-L-proline[c]">'
Missing upper 

Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_43 "allantoin[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_newExRxn_46 "methylmalonate[c]">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_newExRxn_46 "methylmalonate[c]">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_CancerBiomass_Ex "CancerBiomass_ExchangeRxn">'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_CancerBiomass_Ex "CancerBiomass_ExchangeRxn">'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_2391>'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_2391>'
Missing lower flux bound set to '-1000.0' for  reaction: '<Reaction R_HMR_8218>'
Missing upper flux bound set to '1000.0' for  reaction: '<Reaction R_HMR_8218>'
No objective coefficients in model. Unclear what should be optimized
Missing flux bounds on reactions set to default bounds.As best practise

### 5 - Performing a single_reaction_deletion analysis [facultative step] 

In [8]:
if Param_Percentage < 1 :
    # rates = cobra.flux_analysis.single_reaction_deletion(model)
    # or naive ré-implementation of the srd analysis if the cobra function fails
    
    rates = dict()
    for r in model.reactions:
        meml = r.lower_bound
        memu = r.upper_bound
        r.lower_bound = 0
        r.upper_bound = 0
        rates[r.id] = model.optimize().objective_value
        r.lower_bound = meml
        r.upper_bound = memu
else:
    rates = dict()
    for r in model.reactions:
        rates[r.id] = 0
        


### 6 - Get the regulators (Target gene list)

In [12]:
# Objective reference value
if Param_Percentage < 1 :
    fba = model.optimize().objective_value
else:
    fba = 1
    
def getRegulators(rates, percentage=1.1):
    regulators = dict()
    cpt=0
    for r in rates:
        if (rates.get(r)<percentage*fba):
            if (HMR_to_Ensembl.get(r)!=''):
                cpt=cpt+1
                l=HMR_to_Ensembl.get(r)
                if (l):
                    for i in l:
                        if (i != ''):
                            regulators[i]=1
    return regulators

# Targets (Ensembl Ids)
TargetGenesEnsembl = getRegulators(rates, Param_Percentage)

TargetGenes=list()
for x in TargetGenesEnsembl:
    if (genelist.get(x)):
        TargetGenes.append(genelist.get(x)[0])
    else:
        print('Missing ',x)


TargetGenes=list(set(TargetGenes))
TargetGenesEnsembl=list(set(TargetGenesEnsembl))
    
print("Target gene list size =",len(TargetGenes))



# for the sake of clarity and considering that Step 2 possesses a major interest for its own,
# we decided to store the list of target genes between Step 1 and Step 2

with open('../data/target_genes.csv', 'w') as csv_file:
    for g in TargetGenes:
        if g:
            csv_file.write(g+"\n")
csv_file.close()

Missing  ENSG00000123009
Target gene list size = 39


## Step 2 : Drug screening

Our goal is now to demonstrate how pyBravo can support in silico drug screening.

We show here how a list of potential drug targets can be completed based on a *reconstructed gene regulation network*.

We consider that evaluating the biological or clinical relevance of the retrieved potential drugs is out of the scope of this demonstration.


### 0 - Facultative step : pyBravo installation

In [13]:
# required libraries
!pip3 install rdflib requests matplotlib jupyter networkx flask
!pip3 install nxp

# pyBravo
!git clone https://github.com/albangaignard/pyBravo.git

     |████████████████████████████████| 8.5MB 3.6MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/83/df/0f5dd132200728a86190397e1ea87cd76244e42d39ec5e88efd25b2abd7e/jupyter-1.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d2/e07d3ebb2bd7af696440ce7e754c59dd546ffe1bbe732c8ab68b9c834e61/cycler-0.10.0-py2.py3-none-any.whl
     |████████████████████████████████| 2.2MB 11.0MB/s eta 0:00:01     |█▉                              | 122kB 11.0MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/81/c5/9831f281c6fb57945e83fdf39ff036cacfdc84aa1988bb3150b330533050/kiwisolver-1.2.0-cp37-cp37m-macosx_10_9_x86_64.whl
     |████████████████████████████████| 122kB 11.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/56/a0/dbcf5881bb2f51e8db678211907f16ea0a182b232c591a6d6f276985ca95/ipywidgets-7.5.1-py2.py3-none-any.whl
     |████████████████████████████████| 122kB 11.4MB/s eta 0:00:01
     |██████████████

  Using cached https://files.pythonhosted.org/packages/1b/9e/1a170feaa54f22aeb5a5d16c9015e82234275a3c8ab630b552493f9cb8a9/testpath-0.4.4-py2.py3-none-any.whl
     |████████████████████████████████| 153kB 11.4MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/09/ec/4b43dae793655b7d8a25f76119624350b4d65eb663459eb9603d7f1f0345/mistune-0.8.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ac/c6/44694103f8c221443ee6b0041f69e2740d89a25641e62fb4f2ee568f2f9c/entrypoints-0.3-py2.py3-none-any.whl
     |████████████████████████████████| 71kB 10.7MB/s eta 0:00:01
Processing /Users/gaignard-a/Library/Caches/pip/wheels/39/01/56/f1b08a6275acc59e846fa4c1e1b65dbc1919f20157d9e66c20/pandocfilters-1.4.2-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/06/74/9b387472866358ebc08732de3da6dc48e44b0aacd2ddaa5cb85ab7e986a2/defusedxml-0.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/39/7b/88dbb785881c28a1026

In [45]:
# Imports
import csv
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, Markdown, Latex

sparql_endpoint = SPARQLWrapper("https://bio2rdf.org/sparql")

def get_bio2rdf_drug_targets(gene_set):
    """
    From a list of genes (HGNC symbols) this function queries the Bio2RDF sparql endpoint 
    and checks if the symbols are known as a drug target (Drugbank). It returns the list of drug targets. 
    """
    drug_targets = set() 
    for g in gene_set:
        ask_query = 'ASK { ?drug <http://bio2rdf.org/drugbank_vocabulary:target> ?target . ?target <http://bio2rdf.org/drugbank_vocabulary:gene-name> "'+g+'"^^xsd:string}'

        sparql_endpoint.setQuery(ask_query)
        sparql_endpoint.setReturnFormat(JSON)
        results = sparql_endpoint.query().convert()
        if (results["boolean"] == True):
            print(f'found drug targeting {g} in Drugbank (live Bio2RDF endpoint)')
            drug_targets.add(g)
    
    return drug_targets

def get_bio2rdf_drugs(gene_set):
    """
    From a list of genes (HGNC symbols) known as drug targets, returns the URL of found drug in Drugbank. 
    """
    drugs = set() 
    drug_groups = ['Approved', 'Nutraceutical', 'Experimental']
    group = drug_groups[0]
    for g in gene_set:
        query = 'SELECT ?drug ?name ?indication_label WHERE { \
            ?drug <http://bio2rdf.org/drugbank_vocabulary:target> ?target .\
            ?target <http://bio2rdf.org/drugbank_vocabulary:gene-name> "'+g+'"^^xsd:string .\
            ?drug rdf:type <http://bio2rdf.org/drugbank_vocabulary:Drug> .\
            ?drug <http://bio2rdf.org/drugbank_vocabulary:group> <http://bio2rdf.org/drugbank_vocabulary:'+group+'> .\
            ?drug <http://purl.org/dc/terms/title> ?name .\
            ?drug <http://bio2rdf.org/drugbank_vocabulary:indication> ?indication .\
            ?indication <http://purl.org/dc/terms/description> ?indication_label . \
            }'

        sparql_endpoint.setQuery(query)
        sparql_endpoint.setReturnFormat(JSON)
        results = sparql_endpoint.query().convert()
        
        for r in results["results"]["bindings"]:
            d = r["drug"]["value"]
            d_url = d.replace("http://bio2rdf.org/drugbank:", "https://www.drugbank.ca/drugs/")
            ind = r["indication_label"]["value"]
            name = r["name"]["value"]
            print(f'{g} --> {d_url}')
            drugs.add((d_url, name, ind))
    
    return drugs

### 1 - Loading input gene list to be considered as drug targets

In [30]:
# Read input genes list from a file
def read_input_genes(filename):
    res = set()
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in reader:
            #print(''.join(row))
            res.add(''.join(row))
    return res

# Genes input list
gene_set = read_input_genes('../data/target_genes.csv')

#gene_set = [g for g in TargetGenes if g is not None]
print('{} target genes'.format(len(gene_set)))

39 target genes


### 2 - Retrieving potential drugs

In [21]:
%%time
#drug_targets = get_bio2rdf_drug_targets(gene_set)
#potential_drugs = get_bio2rdf_drugs(drug_targets)

potential_drugs = get_bio2rdf_drugs(gene_set)

SPTLC1 --> https://www.drugbank.ca/drugs/DB00133
SPTLC2 --> https://www.drugbank.ca/drugs/DB00133
CPU times: user 397 ms, sys: 37.8 ms, total: 435 ms
Wall time: 5.05 s


In [22]:
display(Markdown(f"**{len(potential_drugs)}** found **potential drugs** in bio2rdf"))

**1** found **potential drugs** in bio2rdf

### 3 - Reconstructing the gene regulation network with pyBravo (max depth = 1)

In [23]:
!python pyBravo/src/pyBravo.py --regulation --input_file ../data/target_genes.csv -md 1 -excl mirtarbase msigdb -co -su -sy

Explored 3 regulators
Explored 3 regulators
Explored 3 regulators
Explored 6 regulators
Explored 10 regulators
Explored 10 regulators
Explored 10 regulators
Explored 11 regulators
Explored 13 regulators
Explored 13 regulators
Explored 15 regulators
Explored 16 regulators
Explored 16 regulators
Explored 16 regulators
Explored 16 regulators
Explored 16 regulators
Explored 16 regulators
Explored 16 regulators
Explored 16 regulators
Explored 16 regulators
Explored 16 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 17 regulators
Explored 18 regulators
Explored 18 reg

### 4 - Extracting the gene regulators to be considered as potential drug targets

In [24]:
def read_regulatory_genes(filename):
    res = set()
    with open(filename, newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter='\t', quotechar='|')
        for row in reader:
            for item in row:
                res.add(item)
    return res

reg_gene_set = read_regulatory_genes('out-unified.sif')
#reg_gene_set = read_regulatory_genes('out.sif')

to_be_explored_genes = reg_gene_set - gene_set

print('{} new potential target genes'.format(len(to_be_explored_genes)))

38 new potential target genes


### 5 - Retrieving new potential drug targets

In [47]:
%%time
#drug_targets = get_bio2rdf_drug_targets(gene_set)
#potential_drugs = get_bio2rdf_drugs(drug_targets)

new_potential_drugs = get_bio2rdf_drugs(to_be_explored_genes)

TGFB1 --> https://www.drugbank.ca/drugs/DB00070
VEGFA --> https://www.drugbank.ca/drugs/DB00112
VEGFA --> https://www.drugbank.ca/drugs/DB01120
VEGFA --> https://www.drugbank.ca/drugs/DB01136
VEGFA --> https://www.drugbank.ca/drugs/DB06779
VEGFA --> https://www.drugbank.ca/drugs/DB01017
VEGFA --> https://www.drugbank.ca/drugs/DB05294
VEGFA --> https://www.drugbank.ca/drugs/DB01270
VEGFA --> https://www.drugbank.ca/drugs/DB08885
TNF --> https://www.drugbank.ca/drugs/DB00005
TNF --> https://www.drugbank.ca/drugs/DB00051
TNF --> https://www.drugbank.ca/drugs/DB00065
TNF --> https://www.drugbank.ca/drugs/DB00608
TNF --> https://www.drugbank.ca/drugs/DB01041
TNF --> https://www.drugbank.ca/drugs/DB01296
TNF --> https://www.drugbank.ca/drugs/DB06674
TNF --> https://www.drugbank.ca/drugs/DB08904
TNF --> https://www.drugbank.ca/drugs/DB08910
TNF --> https://www.drugbank.ca/drugs/DB00668
TNF --> https://www.drugbank.ca/drugs/DB00852
TNF --> https://www.drugbank.ca/drugs/DB01407
TNF --> https://

In [48]:
display(Markdown(f"**{len(new_potential_drugs)}** found **potential drugs** in bio2rdf"))

**46** found **potential drugs** in bio2rdf

In [49]:
out_md = """
| Drug | Name | Indication |
|-------|-----|----| 
"""
for d in new_potential_drugs:
    out_md += "| " + d[0] + " | " + d[1] + " | " + d[2].replace("\r","").replace("\n","") + " |\n"
    
display(Markdown(out_md))


| Drug | Name | Indication |
|-------|-----|----| 
| https://www.drugbank.ca/drugs/DB08885 | Aflibercept | The opthalmic agent is used for the treatment of neovascular (wet) age-related mascular degeneration (AMD) and macular edema following central retinal vein occulsion (CRVO). The systemic injection, known as ziv-aflibercept, in combination with 5-fluorouracil, leucovorin, irinotecan-(FOLFIRI), is for the treatment of metastatic colorectal cancer that is resistant to or progressed following treatment with oxaliplatin. |
| https://www.drugbank.ca/drugs/DB00783 | Estradiol | For the treatment of urogenital symptoms associated with post-menopausal atrophy of the vagina (such as dryness, burning, pruritus and dyspareunia) and/or the lower urinary tract (urinary urgency and dysuria). |
| https://www.drugbank.ca/drugs/DB01407 | Clenbuterol | Used as a bronchodilator in the treatment of asthma patients. |
| https://www.drugbank.ca/drugs/DB00977 | Ethinyl Estradiol | For treatment of moderate to severe vasomotor symptoms associated with the menopause, female hypogonadism, prostatic carcinoma-palliative therapy of advanced disease, breast cancer, as an oral contraceptive, and as emergency contraceptive. |
| https://www.drugbank.ca/drugs/DB00005 | Etanercept | For treatment of severe active rheumatoid arthritis in adults, severe juvenile idiopathic arthritis, ankylosing spondylitis, and severe plaque psoriasis. |
| https://www.drugbank.ca/drugs/DB01393 | Bezafibrate | For the treatment of primary hyperlipidaemia types IIa, IIb, III, IV and V (Fredrickson classification) corresponding to groups I, II and III of the European Atherosclerosis Society guidelines - when diet alone or improvements in lifestyle such as increased exercise or weight reduction do not lead to an adequate response. Also for the treatment of secondary hyperlipidaemias, e.g. severe hypertriglyceridemias, when sufficient improvement does not occur after correction of the underlying disorder (e.g. diabetes mellitus). |
| https://www.drugbank.ca/drugs/DB01270 | Ranibizumab | For the treatment of patients with macular edema after retinal vein occlusion, age-related macular degeneration (wet), and diabetic macular edema. |
| https://www.drugbank.ca/drugs/DB01427 | Amrinone | Used in the treatment of congestive heart failure. |
| https://www.drugbank.ca/drugs/DB00668 | Epinephrine | Used to treat anaphylaxis and sepsis. Also one of the body's main adrenergic neurotransmitters. |
| https://www.drugbank.ca/drugs/DB00608 | Chloroquine | For the suppressive treatment and for acute attacks of malaria due to P. vivax, P.malariae, P. ovale, and susceptible strains of P. falciparum, Second-line agent in treatment of Rheumatoid Arthritis |
| https://www.drugbank.ca/drugs/DB01017 | Minocycline | For the treatment of infections caused by susceptible strains of microorganisms, such as Rocky Mountain spotted fever, typhus fever and the typhus group, Q fever, rickettsial pox and tick fevers caused by Rickettsiae, upper respiratory tract infections caused by <i>Streptococcus pneumoniae</i> and for the treatment of asymptomatic carriers of <i>Neisseria meningitidis</i>. |
| https://www.drugbank.ca/drugs/DB01045 | Rifampicin | For the treatment of Tuberculosis and Tuberculosis-related mycobacterial infections. |
| https://www.drugbank.ca/drugs/DB00481 | Raloxifene | For the prevention and treatment of osteoporosis in post-menopausal women, as well as prevention and treatment of corticosteroid-induced bone loss. Also for the reduction in the incidence of invasive breast cancer in postmenopausal women with osteoporosis or have a high risk for developing breast cancer. |
| https://www.drugbank.ca/drugs/DB01108 | Trilostane | Used in the treatment of Cushing's syndrome. It is normally used in short-term treatment until permanent therapy is possible. |
| https://www.drugbank.ca/drugs/DB00852 | Pseudoephedrine | For the treatment of nasal congestion, sinus congestion, Eustachian tube congestion, and vasomotor rhinitis, and as an adjunct to other agents in the optimum treatment of allergic rhinitis, croup, sinusitis, otitis media, and tracheobronchitis. Also used as first-line therapy of priapism. |
| https://www.drugbank.ca/drugs/DB01229 | Paclitaxel | Used in the treatment of Kaposi's sarcoma and cancer of the lung, ovarian, and breast. Abraxane® is specfically indicated for the treatment of metastatic breast cancer and locally advanced or metastatic non-small cell lung cancer. |
| https://www.drugbank.ca/drugs/DB00530 | Erlotinib | For the treatment of patients with locally advanced or metastatic non-small cell lung cancer after failure of at least one prior chemotherapy regimen. Also for use, in combination with gemcitabine, as the first-line treatment of patients with locally advanced, unresectable or metastatic pancreatic cancer. |
| https://www.drugbank.ca/drugs/DB08904 | Certolizumab pegol | Reducing signs and symptoms of Crohn's disease and treatment of moderately to severely active rheumatoid arthritis (RA). |
| https://www.drugbank.ca/drugs/DB01220 | Rifaximin | For the treatment of patients (≥12 years of age) with travelers' diarrhea caused by noninvasive strains of <i>Escherichia coli</i>. Rifaximin is also designated an orphan drug by the Food and Drug Administration for the adjunctive treatment of hepatic encephalopathy to reduce blood ammonia concentrations and decrease severity of neurological manifestations. |
| https://www.drugbank.ca/drugs/DB04573 | Estriol | Used as a test to determine the general health of an unborn fetus. |
| https://www.drugbank.ca/drugs/DB00328 | Indomethacin | For moderate to severe rheumatoid arthritis including acute flares of chronic disease, ankylosing spondylitis, osteoarthritis, acute painful shoulder (bursitis and/or tendinitis) and acute gouty arthritis. |
| https://www.drugbank.ca/drugs/DB08864 | Rilpivirine | Treatment of HIV-1 infections in treatment-naive patients with HIV-1 RNA ≤100,000 copies/mL in combination with at least 2 other antiretroviral agents. |
| https://www.drugbank.ca/drugs/DB01296 | Glucosamine | Glucosamine is usually used in the treatment of osteoarthritis, although its efficacy is still in question. |
| https://www.drugbank.ca/drugs/DB06779 | Dalteparin | Dalteparin is used as a prophylaxis for deep-vein thrombosis and pulmonary embolisms in patients undergoing general surgery (e.g., abdominal, gynecologic, urologic), and in patients with acute medical conditions (e.g. cancer, bed rest, heart failure, severe lung disease). It is also used in patients who have severely restricted mobility, which poses a risk for thromboembolic complications. Dalteparin is also used concomitantly with aspirin and/or other therapy (e.g., nitrates, β-adrenergic blockers, clopidogrel, platelet glycoprotein [GP] IIb/IIIa-receptor inhibitors) to reduce the risk of acute cardiac ischemic events. The patients who undergo this treatment combination have unstable angina or non-ST-segment elevation/non-Q-wave myocardial infarction (i.e., non-ST-segment elevation acute coronary syndromes).It is also used in the prevention of clotting during hemodialysis and hemofiltration in connection with acute renal failure or chronic renal insufficiency. |
| https://www.drugbank.ca/drugs/DB01136 | Carvedilol | For the treatment of mild or moderate (NYHA class II or III) heart failure of ischemic or cardiomyopathic origin. |
| https://www.drugbank.ca/drugs/DB01120 | Gliclazide | For the treatment of NIDDM in conjunction with diet and exercise. |
| https://www.drugbank.ca/drugs/DB00051 | Adalimumab | For treatment of rheumatoid arthritis, psoriatic arthritis, ankylosing spondylitis, and Crohn's disease. |
| https://www.drugbank.ca/drugs/DB01411 | Pranlukast | Used as an adjunct to the standard therapy of inhaled steroids with inhaled long- and/or short-acting beta-agonists. |
| https://www.drugbank.ca/drugs/DB01041 | Thalidomide | For the acute treatment of the cutaneous manifestations of moderate to severe erythema nodosum leprosum (ENL). Also for use as maintenance therapy for prevention and suppression of the cutaneous manifestations of ENL recurrence. |
| https://www.drugbank.ca/drugs/DB06674 | golimumab | Used in adults (i) as an adjunct to methotrexate treatment in patients with moderate to severe active rheumatoid arthritis (RA), (ii) alone or as an adjunct to methotrexate treatment in patients with active psoriatic arthritis (PsA), (iii) as a single agent in patients with active ankylosing spondylitis (AS), and (iv) as a single agent in patients with moderate to severe ulcerative colitis (UC) who require chronic steroids or have experienced intolerance or only a partial response to previous medications. |
| https://www.drugbank.ca/drugs/DB00065 | Infliximab | To manage the signs and symptoms, as well as to induce and maintain clinical remission in adults with moderate to severe active Crohn's disease or ulcerative colitis. Also used to manage signs and symptoms of rheumatoid arthritis (in conjunction with methotrexate), ankylosing spondylitis, psoriatic arthritis, and juvenile arthritis. |
| https://www.drugbank.ca/drugs/DB00163 | Vitamin E | Vitamin E, known for its antioxidant activities, is protective against cardiovascular disease and some forms of cancer and has also demonstrated immune-enhancing effects. It may be of limited benefit in some with asthma and rheumatoid arthritis. It may be helpful in some neurological diseases including Alzheimer's, some eye disorders including cataracts, and diabetes and premenstrual syndrome. It may also help protect skin from ultraviolet irradiation although claims that it reverses skin aging, enhances male fertility and exercise performance are poorly supported. It may help relieve some muscle cramps. |
| https://www.drugbank.ca/drugs/DB01241 | Gemfibrozil | For treatment of adult patients with very high elevations of serum triglyceride levels (types IV and V hyperlipidemia) who are at risk of developing pancreatitis (inflammation of the pancreas) and who do not respond adequately to a strict diet. |
| https://www.drugbank.ca/drugs/DB01196 | Estramustine | For the palliative treatment of patients with metastatic and/or progressive carcinoma of the prostate |
| https://www.drugbank.ca/drugs/DB00945 | Acetylsalicylic acid | For use in the temporary relief of various forms of pain, inflammation associated with various conditions (including rheumatoid arthritis, juvenile rheumatoid arthritis, systemic lupus erythematosus, osteoarthritis, and ankylosing spondylitis), and is also used to reduce the risk of death and/or nonfatal myocardial infarction in patients with a previous infarction or unstable angina pectoris. |
| https://www.drugbank.ca/drugs/DB05294 | Vandetanib | Vandetanib is currently approved as an alternative to local therapies for both unresectable and disseminated disease. Because Vandetanib can prolong the Q-T interval, it is contraindicated for use in patients with serious cardiac complications such as congenital long QT syndrome and uncompensated heart failure. |
| https://www.drugbank.ca/drugs/DB04574 | Estropipate | Estropipate is used for the treatment of moderate to severe vasomotor symptoms associated with the monopause, and moderate to severe symptoms of vulval and vaginal atrophy associated with the menopause. It is also used to treat hypoestrogenism due to hypogonadism, castration or primary ovarian failure, and prevent postmenopausal osteoporosis. |
| https://www.drugbank.ca/drugs/DB01039 | Fenofibrate | For use as adjunctive therapy to diet to reduce elevated LDL-C, Total-C,Triglycerides and Apo B, and to increase HDL-C in adult patients with primary hypercholesterolemia or mixed dyslipidemia (Fredrickson Types IIa and IIb) |
| https://www.drugbank.ca/drugs/DB00112 | Bevacizumab | As part of combination therapy for metastatic colorectal cancer and HER2-negative metastatic breast cancer. |
| https://www.drugbank.ca/drugs/DB01248 | Docetaxel | For the treatment of patients with locally advanced or metastatic breast cancer after failure of prior chemotherapy. Also used as a single agent in the treatment of patients with locally advanced or metastatic non-small cell lung cancer after failure of prior platinum-based chemotherapy. It is also used in combination with prednisone, in the treatment of patients with androgen independent (hormone refractory) metastatic prostate cancer. Furthermore, docetaxel has uses in the treatment of gastric adenocarinoma and head and neck cancer. |
| https://www.drugbank.ca/drugs/DB00636 | Clofibrate | For Primary Dysbetalipoproteinemia (Type III hyperlipidemia) that does not respond adequately to diet. This helps control high cholesterol and high triglyceride levels. |
| https://www.drugbank.ca/drugs/DB08910 | Pomalidomide | Pomalidomide is indicated for patients with multiple myeloma who have received at least two prior therapies including lenalidomide and bortezomib and have demonstrated disease progression on or within 60 days of completion of the last therapy. |
| https://www.drugbank.ca/drugs/DB00255 | Diethylstilbestrol | Used in the treatment of prostate cancer. Previously used in the prevention of miscarriage or premature delivery in pregnant women prone to miscarriage or premature delivery. |
| https://www.drugbank.ca/drugs/DB00675 | Tamoxifen | Tamoxifen is indicated for the treatment of metastatic breast cancer in women and men and ductal carcinoma in Situ. |
| https://www.drugbank.ca/drugs/DB00070 | Hyaluronidase | For increase of absorption and distribution of other injected drugs and for rehydration. |
| https://www.drugbank.ca/drugs/DB08813 | Nadroparin | Nadroparin is used for prophylaxis of thromboembolic disorders and general surgery in orthopedic surgery, treatment of deep vein thrombosis, prevention of clotting during hemodialysis and treatment of unstable angina and non-Q wave myocardial infarction |
